<a href="https://colab.research.google.com/github/waytogoasis/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import requests
from scipy import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    accuracy_score,
    confusion_matrix,
    classification_report
)
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# Constantes e configurações
DRIVE_MOUNT_PATH = "/content/drive"
GRAPHPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "graficos")
SUMMARYPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "sumarios")
DEFAULT_PALETTE = "viridis"
FIGSIZE = (12, 8)

# Configuração de estilo para gráficos (FUNDO PRETO)
sns.set(style="darkgrid")  # Usaremos darkgrid como base, e modificaremos
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 22,
    'axes.titlesize': 18,
    'axes.labelsize': 14,
})

# Função para carregar os dados
def load_data(sheet_url):
    try:
        response = requests.get(sheet_url)
        response.raise_for_status()
        csv_content = response.content.decode('utf-8')
        csv_file = io.StringIO(csv_content)
        df = pd.read_csv(csv_file)
        return df
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None

def save_fig(filename):
    filepath = os.path.join(GRAPHPATH, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()

def create_figure():
    return plt.figure(figsize=FIGSIZE)

# --- Funções de Visualização SIMPLIFICADAS ---

def generate_boxplot(df, x, y, title="", filename="", palette=None):
    """Gera um boxplot."""
    if not all(col in df.columns for col in [x, y]):  return
    create_figure()
    if palette is None: palette = DEFAULT_PALETTE
    sns.boxplot(x=x, y=y, data=df, palette=palette)
    plt.title(title)
    plt.xticks(rotation=45, ha="right")
    save_fig(filename)

def generate_scatterplot(df, x, y, title="", filename="", palette=None):
    """Gera um scatterplot."""
    if not all(col in df.columns for col in [x, y]): return
    create_figure()
    if palette is None: palette = DEFAULT_PALETTE
    sns.scatterplot(x=x, y=y, data=df, palette=palette, alpha=0.7)
    plt.title(title)
    save_fig(filename)

def generate_histogram(df, column, title="", filename="", bins=30):
    """Gera um histograma."""
    if column not in df.columns: return
    create_figure()
    if not pd.api.types.is_numeric_dtype(df[column]):
        try: df[column] = pd.to_numeric(df[column])
        except ValueError: return
    sns.histplot(df[column], kde=True, bins=bins, color='skyblue') # Cor fixa
    plt.title(title)
    save_fig(filename)

def generate_barplot(df, x, title="", filename=""):
    """Gera um gráfico de barras."""
    if x not in df.columns: return
    create_figure()
    if pd.api.types.is_numeric_dtype(df[x]):
        df[x] = df[x].astype(str)
    sns.countplot(x=x, data=df, palette='pastel')  # Paleta fixa
    plt.title(title)
    plt.xticks(rotation=45, ha="right")
    save_fig(filename)


# --- Funções de Análise Estatística e Modelagem SIMPLIFICADAS ---

def perform_t_test(df, column1, column2):
    """Teste t."""
    if not all(col in df.columns for col in [column1, column2]): return
    try:
        group1 = df[column1].dropna()
        group2 = df[column2].dropna()
        t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False) # Simplificado
        print(f"Teste t entre {column1} e {column2}: t={t_stat:.3f}, p={p_value:.3f}")
        return t_stat, p_value
    except: return None, None

def perform_linear_regression(df, x_columns, y_column):
    """Regressão linear."""
    if not all(col in df.columns for col in x_columns) or y_column not in df.columns : return
    try:
        X = df[x_columns]
        y = df[y_column]
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        print(model.summary())
        return model
    except: return None


# --- Função Principal (main) ---
if __name__ == "__main__":
    drive.mount(DRIVE_MOUNT_PATH, force_remount=True)
    os.makedirs(GRAPHPATH, exist_ok=True)
    os.makedirs(SUMMARYPATH, exist_ok=True)
    sheet_url = "https://docs.google.com/spreadsheets/d/1IJEGj0It36xR2qJSSSelgSxRP90sOv8pK04Bwvoez4E/export?format=csv"
    df = load_data(sheet_url)
    if df is not None:
        print("Colunas originais:", df.columns)
        column_mapping = {
            'Qual a sua idade?': 'Idade',
            'Qual o seu gênero?': 'Genero',
            'Qual a sua formação?': 'Formacao',
            'Em qual área você atua?': 'AreaAtuacao',
            'Quantos anos de experiência você possui na área?': 'AnosExperiencia',
            'Em uma escala de 1 a 10, qual o nível de dificuldade que você atribui à disciplina de Mecânica?': 'NivelDificuldade',
            'Em uma escala de 1 a 10, como você avalia sua compreensão dos conceitos fundamentais da Mecânica?': 'CompreensaoConceitos',
            'Em uma escala de 1 a 10, como você avalia a aplicabilidade prática dos conceitos aprendidos na disciplina de Mecânica?': 'AplicabilidadePratica',
            'Em uma escala de 1 a 10, qual o seu nível de motivação para estudar Mecânica?': 'Motivacao',
            'Em uma escala de 1 a 10, como você avalia a clareza do professor ao explicar os conteúdos?': 'ClarezaProfessor',
            'Em uma escala de 1 a 10, como você avalia a qualidade do material didático utilizado (livros, apostilas, slides, etc.)?': 'MaterialDidatico',
            'Em uma escala de 1 a 10, como você avalia, de forma geral, a disciplina de Mecânica?': 'AvaliacaoGeral',
            'Você tem algum comentário, crítica ou sugestão sobre a disciplina de Mecânica?': 'Comentarios',
            'Em uma escala de 1 a 10, quanto tempo você dedicou aos estudos de Mecânica, incluindo aulas, leituras, exercícios e projetos?': 'TempoDedicacao',
            'Em uma escala de 1 a 10, como você avalia sua interação com as ferramentas computacionais utilizadas na disciplina (softwares de simulação, modelagem, etc.)?': 'InteracaoFerramentas',
            'Em uma escala de 1 a 10, qual o seu nível de satisfação com as ferramentas computacionais utilizadas na disciplina?': 'SatisfacaoFerramentas',
            'Você utilizou alguma ferramenta computacional (software) para auxiliar nos estudos de Mecânica?': 'UsoFerramentas'
        }
        df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns}, inplace=True)
        print("Colunas após renomear:", df.columns)
        df.drop(columns=['Timestamp', 'Comentarios'], inplace=True, errors='ignore')
        cols_to_numeric = ["Idade", "AnosExperiencia", "NivelDificuldade", "CompreensaoConceitos",
                           "AplicabilidadePratica", "Motivacao", "ClarezaProfessor",
                           "MaterialDidatico", "AvaliacaoGeral", "TempoDedicacao",
                           "InteracaoFerramentas", "SatisfacaoFerramentas"]
        for col in cols_to_numeric:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        df.dropna(subset=cols_to_numeric, inplace=True)
        categorical_cols = ['Genero', 'Formacao', 'AreaAtuacao', 'UsoFerramentas']
        label_encoders = {}
        for col in categorical_cols:
            if col in df.columns:
                df[col] = df[col].astype(str)
                df[col] = df[col].fillna('Desconhecido')
                label_encoders[col] = LabelEncoder()
                df[col] = label_encoders[col].fit_transform(df[col])

        # --- Visualizações SIMPLIFICADAS ---
        generate_boxplot(df, "Formacao", "AvaliacaoGeral", title="Avaliação Geral por Formação", filename="boxplot_avaliacao.png")
        generate_scatterplot(df, "AnosExperiencia", "AvaliacaoGeral", title="Avaliação Geral vs. Anos de Experiência", filename="scatterplot_experiencia_avaliacao.png")
        generate_histogram(df, "Idade", title="Distribuição de Idade", filename="histogram_idade.png")
        generate_barplot(df, 'Genero', title="Distribuição de Gênero", filename="barplot_genero.png")
        generate_barplot(df, 'Formacao', title="Distribuição de Formação", filename="barplot_formacao.png")
        generate_barplot(df, 'AreaAtuacao', title="Distribuição de Área de Atuação", filename="barplot_area.png")
        generate_scatterplot(df, "Motivacao", "AvaliacaoGeral", title="Avaliação Geral vs. Motivação", filename="scatterplot_motivacao_avaliacao.png")
        generate_histogram(df, "AvaliacaoGeral", title="Distribuição de Avaliação Geral", filename="histogram_avaliacao.png")

        # --- Análise Estatística SIMPLIFICADA ---
        perform_t_test(df, "AvaliacaoGeral", "Motivacao")

        # --- Modelagem SIMPLIFICADA ---
        features_regressao = [col for col in df.columns if col != 'AvaliacaoGeral']
        perform_linear_regression(df, features_regressao, "AvaliacaoGeral")

        print("Análise e gráficos gerados com sucesso!")

Mounted at /content/drive
Colunas originais: Index(['Idade', 'AnosExperiencia', 'NivelDificuldade', 'CompreensaoConceitos',
       'AplicabilidadePratica', 'Motivacao', 'ClarezaProfessor',
       'MaterialDidatico', 'AvaliacaoGeral', 'Genero', 'Formacao',
       'AreaAtuacao', 'TempoDedicacao', 'InteracaoFerramentas',
       'SatisfacaoFerramentas', 'UsoFerramentas'],
      dtype='object')
Colunas após renomear: Index(['Idade', 'AnosExperiencia', 'NivelDificuldade', 'CompreensaoConceitos',
       'AplicabilidadePratica', 'Motivacao', 'ClarezaProfessor',
       'MaterialDidatico', 'AvaliacaoGeral', 'Genero', 'Formacao',
       'AreaAtuacao', 'TempoDedicacao', 'InteracaoFerramentas',
       'SatisfacaoFerramentas', 'UsoFerramentas'],
      dtype='object')
Teste t entre AvaliacaoGeral e Motivacao: t=-0.439, p=0.662
Análise e gráficos gerados com sucesso!
